# pixelclassifier tutorial

This notebook illustrates how to use pixelclassifier to highlight cell contours by training a model with 'contour' versus 'non-contour' labels.

---
We start with some general imports.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gpfunctions import generateSynthCellsImage, imshow, imshowlist, \
    imerode, imdilate, tifwrite, im2double, tifread, stack2list, createFolderIfNonExistent
%matplotlib inline

---
We will generate some synthetic data: 5 [image, label] pairs for training, and 1 [image, label] pair for testing. Notice that the labels are 'class balanced', i.e. in each image the number of contour pixels is roughly the same as the number of non-contour pixels.

In [ ]:
train_folder = 'DataForPC/Train'
test_folder = 'DataForPC/Test'
createFolderIfNonExistent(train_folder)
createFolderIfNonExistent(test_folder)

def gen_img_lbl_pair():
    I, L = generateSynthCellsImage()

    cells = imdilate(L > 0, 1)
    interior = imerode(cells, 2)
    contours = np.logical_and(cells, np.logical_not(interior))
    not_contours = np.logical_not(contours)

    c_cont = np.sum(contours)

    f = c_cont/np.sum(not_contours)
    not_contours = np.logical_and(not_contours, np.random.rand(I.shape[0], I.shape[1]) < f)
    
    print('n pixels class 0:', np.sum(contours), '| npixels class 1:', np.sum(not_contours))

    L[:] = 0
    L[contours] = 1
    L[not_contours] = 2
    
    return I, L
    
for idx in range(5):
    print('train image', idx)
    
    I, L = gen_img_lbl_pair()
    tifwrite(np.uint8(255*I), '%s/I%05d_Img.tif' % (train_folder, idx))
    tifwrite(L, '%s/I%05d_Ant.tif' % (train_folder, idx))

for idx in range(1):
    print('test iamge', idx)
    
    I, L = gen_img_lbl_pair()
    tifwrite(np.uint8(255*I), '%s/I%05d_Img.tif' % (test_folder, idx))
    tifwrite(L, '%s/I%05d_Ant.tif' % (test_folder, idx))

---
Let's take a look at the test pair.

In [ ]:
I = tifread('%s/I%05d_Img.tif' % (test_folder, 0))
L = tifread('%s/I%05d_Ant.tif' % (test_folder, 0))

plt.figure(figsize=(12, 6))

plt.subplot(1,2,1)
plt.imshow(I, cmap='gray'); plt.axis('off'); plt.title('image');

plt.subplot(1,2,2)
red = L == 1
green = L == 2
blue = np.zeros(L.shape, L.dtype)
rgb = 255*np.stack([red, green, blue], axis=2)
plt.imshow(rgb); plt.axis('off'); plt.title('labels (red: contour, green: non-contour)');

---
To train a Random Forest model via pixelclassifier we simply pass the path to the training set as well as scale parameters for derivatives and LoG filters. We expect derivatives to be sufficient, so we pass an empty list as the sigmaLoG parameter. Once the model is done training, a plot of feature importances is shown.

In [ ]:
import pixelclassifier as pc
import os

trainPath = 'DataForPC/Train'
model = pc.train(trainPath,sigmaDeriv=[2,4],sigmaLoG=[])

pc.plotFeatImport(model['featImport'],model['featNames'])

---
Let's now see how the model performs on the test image. We can ask the classifier for either class or probability map outputs.

In [ ]:
path = 'DataForPC/Test/I00000_Img.tif'
I = im2double(tifread(path))

C = pc.classify(I,model,output='classes')
P = pc.classify(I,model,output='probmaps')


plt.figure(figsize=(8, 12))

plt.subplot(3,2,1)
plt.imshow(I, cmap='gray'); plt.axis('off'); plt.title('image');

plt.subplot(3,2,3)
plt.imshow(C[:,:,0], cmap='gray'); plt.axis('off'); plt.title('mask class 1');

plt.subplot(3,2,4)
plt.imshow(C[:,:,1], cmap='gray'); plt.axis('off'); plt.title('mask class 2');

plt.subplot(3,2,5)
plt.imshow(P[:,:,0], cmap='gray'); plt.axis('off'); plt.title('prob. map class 1');

plt.subplot(3,2,6)
plt.imshow(P[:,:,1], cmap='gray'); plt.axis('off'); plt.title('prob. map class 2');